<a href="https://colab.research.google.com/github/BharathTT/Machine-learning-models/blob/master/Multilinear%20perceptron_amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bharathskanda","key":"c5b293de1678e1a5d7e2170345a4019b"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d datafiniti/consumer-reviews-of-amazon-products

 31% 5.00M/16.3M [00:00<00:00, 47.4MB/s]
100% 16.3M/16.3M [00:00<00:00, 93.6MB/s]


In [5]:
import zipfile

with zipfile.ZipFile("consumer-reviews-of-amazon-products.zip", "r") as zip_ref:
    zip_ref.extractall("consumer-reviews-of-amazon-products")


In [6]:
ls consumer-reviews-of-amazon-products/

1429_1.csv
Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv
Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv


In [7]:
import pandas as pd
data=pd.read_csv("./consumer-reviews-of-amazon-products/1429_1.csv")

<ipython-input-7-6c4acdc4f19c>:2: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("./consumer-reviews-of-amazon-products/1429_1.csv")


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
data = data.dropna(subset=['reviews.rating', 'reviews.text'])
data['sentiment'] = data['reviews.rating'].apply(lambda x: 1 if x >= 4 else 0)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(data['reviews.text'], data['sentiment'], test_size=0.2, random_state=42)

In [12]:
vectorizer = CountVectorizer(max_features=5000)  # You can adjust max_features as needed
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [13]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary classification, so use sigmoid activation

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [15]:
import tensorflow as tf
import numpy as np
from scipy.sparse import csr_matrix

X_train_csr = csr_matrix(X_train)
X_train_coo = X_train_csr.tocoo()
X_train_sparse = tf.sparse.SparseTensor(
    indices=np.vstack((X_train_coo.row, X_train_coo.col)).T,
    values=X_train_coo.data,
    dense_shape=X_train_coo.shape
)

X_train_sparse_ordered = tf.sparse.reorder(X_train_sparse)

model.fit(X_train_sparse_ordered, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/10
433/433 [==============================] - 8s 15ms/step - loss: 0.2140 - accuracy: 0.9319 - val_loss: 0.1658 - val_accuracy: 0.9401
Epoch 2/10
433/433 [==============================] - 5s 11ms/step - loss: 0.1566 - accuracy: 0.9423 - val_loss: 0.1618 - val_accuracy: 0.9437
Epoch 3/10
433/433 [==============================] - 5s 12ms/step - loss: 0.1341 - accuracy: 0.9528 - val_loss: 0.1702 - val_accuracy: 0.9437
Epoch 4/10
433/433 [==============================] - 5s 11ms/step - loss: 0.1110 - accuracy: 0.9616 - val_loss: 0.1810 - val_accuracy: 0.9443
Epoch 5/10
433/433 [==============================] - 6s 13ms/step - loss: 0.0908 - accuracy: 0.9701 - val_loss: 0.1927 - val_accuracy: 0.9430
Epoch 6/10
433/433 [==============================] - 5s 11ms/step - loss: 0.0744 - accuracy: 0.9755 - val_loss: 0.2186 - val_accuracy: 0.9414
Epoch 7/10
433/433 [==============================] - 5s 11ms/step - loss: 0.0611 - accuracy: 0.9813 - val_loss: 0.2393 - val_accuracy: 0.9415

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')


217/217 [==============================] - 1s 4ms/step - loss: 0.2821 - accuracy: 0.9422
Test Loss: 0.2821, Test Accuracy: 0.9422


In [ ]:
X_new = vectorizer.transform(["This product is great!"])
y_pred = model.predict(X_new)
predicted_sentiment = 1 if y_pred[0] >= 0.5 else 0
print(f'Predicted Sentiment: {"Positive" if predicted_sentiment == 1 else "Negative"}')


1/1 [==============================] - 0s 210ms/step
Predicted Sentiment: Positive


In [ ]:
model.save('sentiment_model1.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
